In [48]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
df_server_1_equal = pd.read_csv('server_one_dataframe_equal_division.csv')

uploaded = files.upload()
df_server_2_equal = pd.read_csv('server_two_dataframe_equal_division.csv')

Saving server_one_dataframe_equal_division.csv to server_one_dataframe_equal_division (1).csv


Saving server_two_dataframe_equal_division.csv to server_two_dataframe_equal_division (1).csv


In [5]:
# CTGAN: https://github.com/sdv-dev/CTGAN
# !pip install ctgan

In [49]:
df_server_1_equal.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.00,01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.00,01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.00,01/06/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.00,01/06/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.00,01/06/2014,6,June,2014


In [50]:
# Gross sales = units solds * sales price
# Gross sales to sales: sales = gross sales - discount
# COGS (cost of goods sold) = units sold * manufacturing price
# profit = sales - COGS
# month name from month number
# private feature is the discount band


# So generate units sold, sales price, discount, manufacturing price, month number, and year

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_1_filtered = df_server_1_equal.drop(columns = columns_to_drop)

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_2_filtered = df_server_2_equal.drop(columns = columns_to_drop)

In [51]:
print(len(df_server_1_filtered))

350


In [52]:
discrete_columns = ['Month Number', 'Year']

from ctgan import CTGAN

ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_1_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [53]:
df_server_1_synthetic = ctgan.sample(len(df_server_1_filtered))

In [54]:
# df_server_1_synthetic.head()
# Now units sold should be an integer, manufacturing, sales price, and discuounts should be > 0
import numpy as np

df_server_1_synthetic['Units Sold'] = np.ceil(df_server_1_synthetic['Units Sold'])
df_server_1_synthetic['Manufacturing Price'] = np.abs(df_server_1_synthetic['Manufacturing Price'])
df_server_1_synthetic['Sale Price'] = np.abs(df_server_1_synthetic['Sale Price'])
df_server_1_synthetic['Discounts'] = np.abs(df_server_1_synthetic['Discounts'])

In [55]:
import calendar

df_server_1_synthetic['Gross Sales'] = df_server_1_synthetic['Units Sold'] * df_server_1_synthetic['Sale Price']
df_server_1_synthetic['Sales'] = df_server_1_synthetic['Gross Sales'] - df_server_1_synthetic['Discounts']
df_server_1_synthetic['COGS'] = df_server_1_synthetic['Units Sold'] * df_server_1_synthetic['Manufacturing Price']
df_server_1_synthetic['Profit'] = df_server_1_synthetic['Sales'] - df_server_1_synthetic['COGS']
df_server_1_synthetic['Month Name'] = df_server_1_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [56]:
df_server_1_synthetic['Segment'] = df_server_1_equal['Segment']
df_server_1_synthetic['Country'] = df_server_1_equal['Country']
df_server_1_synthetic['Product'] = df_server_1_equal['Product']
df_server_1_synthetic['Discount Band'] = df_server_1_equal['Discount Band']
df_server_1_synthetic['Date'] = pd.to_datetime(df_server_1_synthetic['Year'].astype(str) + '-' + df_server_1_synthetic['Month Number'].astype(str) + '-1')
df_server_1_synthetic['Date'] = df_server_1_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [57]:
df_server_1_synthetic = df_server_1_synthetic[df_server_1_equal.columns]

In [58]:
df_server_1_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1341.0,79.773752,20.630323,2.766526e+04,7044.641132,2.062062e+04,106976.601279,-86355.978972,01/05/2013,5,May,2013
1,Government,Germany,Carretera,None,141.0,6.641405,31.357313,4.421381e+03,6518.491783,-2.097111e+03,936.438079,-3033.548792,01/10/2014,10,October,2014
2,Midmarket,France,Carretera,None,566.0,2.771456,123.632733,6.997613e+04,548.361291,6.942777e+04,1568.644196,67859.121434,01/08/2014,8,August,2014
3,Midmarket,Germany,Carretera,None,3286.0,287.201297,327.684416,1.076771e+06,1144.195437,1.075627e+06,943743.462360,131883.332682,01/09/2014,9,September,2014
4,Midmarket,Mexico,Carretera,None,3578.0,1.514401,113.113861,4.047214e+05,2062.489269,4.026589e+05,5418.527629,397240.379214,01/10/2013,10,October,2013


In [60]:
# df_server_1_synthetic.to_csv('server_one_synthetic_dataset_equal.csv', index = False)
# files.download('server_one_synthetic_dataset_equal.csv')

In [61]:
ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_2_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [62]:
df_server_2_synthetic = ctgan.sample(len(df_server_2_filtered))

In [63]:
df_server_2_synthetic['Units Sold'] = np.ceil(df_server_2_synthetic['Units Sold'])
df_server_2_synthetic['Manufacturing Price'] = np.abs(df_server_2_synthetic['Manufacturing Price'])
df_server_2_synthetic['Sale Price'] = np.abs(df_server_2_synthetic['Sale Price'])
df_server_2_synthetic['Discounts'] = np.abs(df_server_2_synthetic['Discounts'])

In [64]:
df_server_2_synthetic['Gross Sales'] = df_server_2_synthetic['Units Sold'] * df_server_2_synthetic['Sale Price']
df_server_2_synthetic['Sales'] = df_server_2_synthetic['Gross Sales'] - df_server_2_synthetic['Discounts']
df_server_2_synthetic['COGS'] = df_server_2_synthetic['Units Sold'] * df_server_2_synthetic['Manufacturing Price']
df_server_2_synthetic['Profit'] = df_server_2_synthetic['Sales'] - df_server_2_synthetic['COGS']
df_server_2_synthetic['Month Name'] = df_server_2_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [65]:
df_server_2_synthetic['Segment'] = df_server_2_equal['Segment']
df_server_2_synthetic['Country'] = df_server_2_equal['Country']
df_server_2_synthetic['Product'] = df_server_2_equal['Product']
df_server_2_synthetic['Discount Band'] = df_server_2_equal['Discount Band']
df_server_2_synthetic['Date'] = pd.to_datetime(df_server_2_synthetic['Year'].astype(str) + '-' + df_server_2_synthetic['Month Number'].astype(str) + '-1')
df_server_2_synthetic['Date'] = df_server_2_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [66]:
df_server_2_synthetic = df_server_2_synthetic[df_server_2_equal.columns]

In [67]:
df_server_2_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Small Business,France,VTT,Medium,573.0,33.730750,381.179701,218415.968804,48007.346517,170408.622287,19327.719502,151080.902785,01/03/2014,3,March,2014
1,Small Business,Mexico,VTT,Medium,1623.0,247.388162,407.110192,660739.841461,134572.635332,526167.206130,401510.987232,124656.218898,01/11/2014,11,November,2014
2,Enterprise,Canada,Amarilla,Medium,1409.0,11.086513,144.671073,203841.541242,4171.285000,199670.256242,15620.897192,184049.359050,01/10/2014,10,October,2014
3,Government,France,Amarilla,Medium,1937.0,19.965613,375.782383,727890.475886,10315.407017,717575.068869,38673.391496,678901.677373,01/07/2014,7,July,2014
4,Enterprise,Germany,Amarilla,Medium,-600.0,82.671740,449.116522,-269469.913023,2803.801197,-272273.714220,-49603.044151,-222670.670069,01/11/2014,11,November,2014


In [68]:
# df_server_2_synthetic.to_csv('server_two_synthetic_dataset_equal.csv', index = False)
# files.download('server_two_synthetic_dataset_equal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
uploaded = files.upload()
df_server_1_north_america = pd.read_csv('server_one_dataframe_north_america.csv')

uploaded = files.upload()
df_server_2_europe = pd.read_csv('server_two_dataframe_europe.csv')

Saving server_one_dataframe_north_america.csv to server_one_dataframe_north_america.csv


Saving server_two_dataframe_europe.csv to server_two_dataframe_europe.csv


In [39]:
columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_1_north_america_filtered = df_server_1_north_america.drop(columns = columns_to_drop)

columns_to_drop = ['Segment', 'Country', 'Product', 'Discount Band', 'Gross Sales', 'Sales', 'COGS', 'Profit', 'Date', 'Month Name']
df_server_2_europe_filtered = df_server_2_europe.drop(columns = columns_to_drop)

In [40]:
discrete_columns = ['Month Number', 'Year']

from ctgan import CTGAN

ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_1_north_america_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [41]:
df_server_1_north_america_synthetic = ctgan.sample(len(df_server_1_north_america_filtered))

In [42]:
df_server_1_north_america_synthetic['Units Sold'] = np.ceil(df_server_1_north_america_synthetic['Units Sold'])
df_server_1_north_america_synthetic['Manufacturing Price'] = np.abs(df_server_1_north_america_synthetic['Manufacturing Price'])
df_server_1_north_america_synthetic['Sale Price'] = np.abs(df_server_1_north_america_synthetic['Sale Price'])
df_server_1_north_america_synthetic['Discounts'] = np.abs(df_server_1_north_america_synthetic['Discounts'])

In [43]:
df_server_1_north_america_synthetic['Gross Sales'] = df_server_1_north_america_synthetic['Units Sold'] * df_server_1_north_america_synthetic['Sale Price']
df_server_1_north_america_synthetic['Sales'] = df_server_1_north_america_synthetic['Gross Sales'] - df_server_1_north_america_synthetic['Discounts']
df_server_1_north_america_synthetic['COGS'] = df_server_1_north_america_synthetic['Units Sold'] * df_server_1_north_america_synthetic['Manufacturing Price']
df_server_1_north_america_synthetic['Profit'] = df_server_1_north_america_synthetic['Sales'] - df_server_1_north_america_synthetic['COGS']
df_server_1_north_america_synthetic['Month Name'] = df_server_1_north_america_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [44]:
df_server_1_north_america_synthetic['Segment'] = df_server_1_north_america['Segment']
df_server_1_north_america_synthetic['Country'] = df_server_1_north_america['Country']
df_server_1_north_america_synthetic['Product'] = df_server_1_north_america['Product']
df_server_1_north_america_synthetic['Discount Band'] = df_server_1_north_america['Discount Band']
df_server_1_north_america_synthetic['Date'] = pd.to_datetime(df_server_1_north_america_synthetic['Year'].astype(str) + '-' + df_server_1_north_america_synthetic['Month Number'].astype(str) + '-1')
df_server_1_north_america_synthetic['Date'] = df_server_1_north_america_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [45]:
df_server_1_north_america_synthetic = df_server_1_north_america_synthetic[df_server_1_north_america.columns]

In [46]:
df_server_1_north_america_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1970.0,258.726455,359.018315,707266.080698,35940.902447,671325.178251,509691.116074,161634.062176,01/01/2014,1,January,2014
1,Midmarket,Mexico,Carretera,None,1002.0,291.937871,136.284332,136556.900851,5056.141995,131500.758855,292521.746507,-161020.987652,01/09/2013,9,September,2013
2,Channel Partners,Canada,Montana,None,2039.0,35.463074,97.555445,198915.551595,136164.915991,62750.635604,72309.207023,-9558.571419,01/06/2014,6,June,2014
3,Midmarket,Mexico,Montana,None,1052.0,149.067149,29.340548,30866.256908,10328.434600,20537.822309,156818.641102,-136280.818794,01/07/2013,7,July,2013
4,Enterprise,Canada,Montana,None,1181.0,32.284835,26.267986,31022.490976,7870.304315,23152.186661,38128.390315,-14976.203655,01/01/2013,1,January,2013


In [47]:
# df_server_1_north_america_synthetic.to_csv('server_one_synthetic_dataset_north_america.csv', index = False)
# files.download('server_one_synthetic_dataset_north_america.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
ctgan = CTGAN(epochs = 10)
ctgan.fit(df_server_2_europe_filtered, discrete_columns)

/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'mi

In [70]:
df_server_2_europe_synthetic = ctgan.sample(len(df_server_2_europe_filtered))

In [71]:
df_server_2_europe_synthetic['Units Sold'] = np.ceil(df_server_2_europe_synthetic['Units Sold'])
df_server_2_europe_synthetic['Manufacturing Price'] = np.abs(df_server_2_europe_synthetic['Manufacturing Price'])
df_server_2_europe_synthetic['Sale Price'] = np.abs(df_server_2_europe_synthetic['Sale Price'])
df_server_2_europe_synthetic['Discounts'] = np.abs(df_server_2_europe_synthetic['Discounts'])

In [72]:
df_server_2_europe_synthetic['Gross Sales'] = df_server_2_europe_synthetic['Units Sold'] * df_server_2_europe_synthetic['Sale Price']
df_server_2_europe_synthetic['Sales'] = df_server_2_europe_synthetic['Gross Sales'] - df_server_2_europe_synthetic['Discounts']
df_server_2_europe_synthetic['COGS'] = df_server_2_europe_synthetic['Units Sold'] * df_server_2_europe_synthetic['Manufacturing Price']
df_server_2_europe_synthetic['Profit'] = df_server_2_europe_synthetic['Sales'] - df_server_2_europe_synthetic['COGS']
df_server_2_europe_synthetic['Month Name'] = df_server_2_europe_synthetic['Month Number'].apply(lambda x: calendar.month_name[x])

In [73]:
df_server_2_europe_synthetic['Segment'] = df_server_2_europe['Segment']
df_server_2_europe_synthetic['Country'] = df_server_2_europe['Country']
df_server_2_europe_synthetic['Product'] = df_server_2_europe['Product']
df_server_2_europe_synthetic['Discount Band'] = df_server_2_europe['Discount Band']
df_server_2_europe_synthetic['Date'] = pd.to_datetime(df_server_2_europe_synthetic['Year'].astype(str) + '-' + df_server_2_europe_synthetic['Month Number'].astype(str) + '-1')
df_server_2_europe_synthetic['Date'] = df_server_2_europe_synthetic['Date'].dt.strftime('%d/%m/%Y')

In [74]:
df_server_2_europe_synthetic = df_server_2_europe_synthetic[df_server_2_europe.columns]

In [75]:
df_server_2_europe_synthetic.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Germany,Carretera,None,2187.0,205.575004,109.131289,238670.129769,51616.455186,187053.674583,449592.533984,-262538.859401,01/11/2013,11,November,2013
1,Midmarket,France,Carretera,None,2789.0,13.850231,102.422008,285654.980462,4775.381853,280879.598610,38628.294815,242251.303794,01/05/2013,5,May,2013
2,Midmarket,Germany,Carretera,None,1539.0,258.691977,9.155531,14090.362240,8431.185341,5659.176899,398126.953060,-392467.776161,01/07/2013,7,July,2013
3,Government,Germany,Carretera,None,1485.0,202.466321,7.401625,10991.412399,4108.344187,6883.068212,300662.486582,-293779.418370,01/11/2013,11,November,2013
4,Midmarket,Germany,Montana,None,3488.0,17.245425,253.267329,883396.442161,29616.089988,853780.352173,60152.042171,793628.310002,01/02/2013,2,February,2013


In [77]:
# df_server_2_europe_synthetic.to_csv('server_two_synthetic_dataset_europe.csv', index = False)
# files.download('server_two_synthetic_dataset_europe.csv')